In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
print(tf.__version__)

2.17.0


In [3]:
df = pd.read_csv("recruitment_data.csv")

In [4]:
df.head()

,Age,Gender,EducationLevel,ExperienceYears,PreviousCompanies,DistanceFromCompany,InterviewScore,SkillScore,PersonalityScore,RecruitmentStrategy,HiringDecision
0,26,1,2,0,3,26.783828,48,78,91,1,1
1,39,1,4,12,3,25.862694,35,68,80,2,1
2,48,0,2,3,2,9.920805,20,67,13,2,0
3,34,1,2,5,2,6.407751,36,27,70,3,0
4,30,0,1,6,1,43.105343,23,52,85,2,0


In [5]:
X = df.drop(['HiringDecision'], axis=1)
y = df['HiringDecision']


Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.3,random_state=42)

scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.transform(Xtest)

Xtrain

array([[ 0.22038632,  1.01729235, -0.23894432, ...,  0.47422282,
         0.55983859,  0.14678193],
       [-0.32028106, -0.98300159, -0.23894432, ...,  0.13709759,
         1.33963452, -1.2936015 ],
       [ 0.11225285,  1.01729235, -0.23894432, ..., -1.1776908 ,
        -0.35557403, -1.2936015 ],
       ...,
       [ 1.30172109, -0.98300159, -0.23894432, ..., -0.3011652 ,
         0.18689271,  0.14678193],
       [ 1.40985457, -0.98300159, -0.23894432, ..., -0.23374016,
         0.62764693,  0.14678193],
       [ 0.00411937, -0.98300159, -1.38982295, ...,  0.30566021,
         1.57696372, -1.2936015 ]])

In [6]:
Xtrain.shape, ytrain.shape,Xtest.shape,ytest.shape

((1050, 10), (1050,), (450, 10), (450,))

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [8]:
model = Sequential([
    Dense(64,activation='relu', input_shape=(Xtrain.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1,activation='sigmoid')
])

c:\Users\pwal9\Desktop\ann\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,817 (11.00 KB)

 Trainable params: 2,817 (11.00 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(optimizer=opt,loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
logs = 'logs/fit/'+datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback= TensorBoard(log_dir=logs,histogram_freq=1)

In [13]:
Erly_stopping = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [14]:
history = model.fit(
    Xtrain,ytrain,validation_data=(Xtest,ytest),epochs=100,
    callbacks=[tensorflow_callback,Erly_stopping]
)

Epoch 1/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7295 - loss: 0.5217 - val_accuracy: 0.8422 - val_loss: 0.3956
Epoch 2/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8795 - loss: 0.3132 - val_accuracy: 0.8511 - val_loss: 0.3890
Epoch 3/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8778 - loss: 0.3308 - val_accuracy: 0.8800 - val_loss: 0.3684
Epoch 4/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8886 - loss: 0.2915 - val_accuracy: 0.8578 - val_loss: 0.3666
Epoch 5/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8998 - loss: 0.2733 - val_accuracy: 0.8711 - val_loss: 0.3702
Epoch 6/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8974 - loss: 0.2731 - val_accuracy: 0.8556 - val_loss: 0.4089
Epoch 7/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8920 - loss: 0.3007 - val_accuracy: 0.8644 - val_loss: 0.3693
Epoch 8/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8942 - loss: 0.2647 - val_accuracy: 0.8778 - v

In [15]:
model.save('model.h5')

In [16]:
import tensorboard


In [17]:
from tensorflow.keras.models import load_model

In [18]:
model1 = load_model('model.h5')

In [19]:
df.head()

,Age,Gender,EducationLevel,ExperienceYears,PreviousCompanies,DistanceFromCompany,InterviewScore,SkillScore,PersonalityScore,RecruitmentStrategy,HiringDecision
0,26,1,2,0,3,26.783828,48,78,91,1,1
1,39,1,4,12,3,25.862694,35,68,80,2,1
2,48,0,2,3,2,9.920805,20,67,13,2,0
3,34,1,2,5,2,6.407751,36,27,70,3,0
4,30,0,1,6,1,43.105343,23,52,85,2,0


In [20]:
a = [48,0,2,3,2,9.92,20,67,13,2]
inp = pd.DataFrame([a])
a

[48, 0, 2, 3, 2, 9.92, 20, 67, 13, 2]

In [21]:
inp = scaler.transform(inp)
inp

c:\Users\pwal9\Desktop\ann\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[ 1.40985457, -0.98300159, -0.23894432, -1.00288435, -0.70744883,
        -1.08325171, -1.09690429,  0.54164787, -1.23708248,  0.14678193]])

In [22]:
prediction = model1.predict(inp)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


array([[0.05610574]], dtype=float32)

In [23]:
prediciton_proba = prediction[0][0]
prediciton_proba

0.056105737